In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

<ipython-input-3-d57511d72f78>:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [ ]:
file_path = "data set path"
main_df = pd.read_csv(file_path)

In [ ]:
df = main_df.copy()
df.head()

In [ ]:
print(df.columns)

In [ ]:
#  make a columb average price and fill it
df["avg"] = ((df["HIGH "]+df["LOW "]) / 2)
df.head(-1)

In [ ]:
df['Next_two_days_avg'] = (df['avg'].shift(-1) + df['avg'].shift(-2)) / 2
df_clean = df.dropna()
df_clean.head(-2)

In [ ]:
# Calculate the percentage change between 'Next_two_days_avg' and 'avg'
percentage_change = (df_clean['Next_two_days_avg'] / df_clean['avg'] - 1) * 100

# Create a new column 'IncreaseDecrease' based on the percentage changed condition
df_clean['IncreaseDecrease'] = np.where(percentage_change > 0.5, 1, 0)
df_clean.head(20)

In [ ]:
# List of column names to convert
columns_to_convert = ['OPEN ', 'HIGH ', 'LOW ', 'PREV. CLOSE ', 'ltp ',
       'close ', 'vwap ', '52W H ', '52W L ', 'VOLUME ',
       'No of trades ']

# Applying the operation to each column
for col in columns_to_convert:
    df_clean[col] = df_clean[col].astype(str).str.replace(',', '').astype(float).astype(int)

In [ ]:
X = df_clean[['HIGH ', 'LOW ', 'PREV. CLOSE ',
        '52W H ', '52W L ',
       'No of trades ']]
Y = df_clean['IncreaseDecrease']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, train_size=0.9, test_size=0.1, random_state=0)

In [ ]:
model = keras.Sequential([
    layers.BatchNormalization(input_shape=[6]),
    layers.Dense(400, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(100, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(10, activation='relu'),
    layers.Dense(1),
])


In [ ]:
custom_optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=custom_optimizer,
    loss='mean_squared_error',
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=16,
    epochs=25,
    verbose=1,
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

In [ ]:
y_pred = model.predict(X_valid)
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error:", mse)

1/1 [==============================] - 0s 77ms/step
Mean Squared Error: 0.188096145839304


In [ ]:
# dumy program to create short term investing enviroment

investment_amount = 200  # Amount to invest

# Initialize variables to track investment performance
total_invested = 0
current_balance = 0  # No initial balance

for i in range(len(y_pred)):
    prediction = int(y_pred[i] > 0.4)  # Assuming a threshold of 0.5 for binary classification

    if prediction == 1:
        sec_prediction = df_clean['Next_two_days_avg'].iloc[i]+20

        # Invest 200
        total_invested += investment_amount 
        result = (0.3 / 100 * df_clean['avg'].iloc[i])+df_clean['avg'].iloc[i]
        print(result)
        print(sec_prediction)
        current_balance = current_balance+(sec_prediction-result)


# Print the results
print("Total Invested:", total_invested)
print("Current Balance:", current_balance)


In [ ]:
# to save the trained model
model.save('v9_model')